In [7]:
import datetime

start_date = datetime.datetime(1997, 6, 16)
end_date = datetime.datetime.now()

start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

In [ ]:
import yfinance as yf

# Scarica i dati per l'indice S&P 500
sp500 = yf.Ticker("^GSPC")

# Dati storici dell'S&P 500 (per esempio, degli ultimi 5 anni)
data = sp500.history(period="max")
#filtro i dati e tengo solo 1 per mese
data = data.resample('ME').last()

# scrivo su csv
data.to_csv('sp500.csv')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. URL della pagina Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"  # Sostituisci con l'URL della pagina di interesse

# 2. Richiesta HTTP alla pagina
response = requests.get(url)
if response.status_code == 200:
    # 3. Parsing del contenuto HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # 4. Trovare la tabella specifica
    # Cerca la tabella basandoti su una classe CSS o un indice se ci sono più tabelle
    table = soup.find("table", {"id": "constituents"})  # 'wikitable' è comune nelle tabelle di Wikipedia

    # 5. Estrazione dei dati dalla tabella
    rows = []
    headers = [header.text.strip() for header in table.find_all("th")]  # Estrae le intestazioni
    for row in table.find_all("tr")[1:]:  # Salta la riga delle intestazioni
        cells = row.find_all(["td", "th"])  # Prende sia td che th, nel caso di celle di intestazione
        rows.append([cell.text.strip() for cell in cells])

    # 6. Converti i dati in un DataFrame di Pandas (opzionale ma utile)
    df = pd.DataFrame(rows, columns=headers)
    #print(df)
    # 7. Salva i dati in un file CSV
    df.to_csv("sp500_companies.csv", index=False)

    table = soup.find("table", {"id": "changes"})  # 'wikitable' è comune nelle tabelle di Wikipedia

    # 5. Estrazione dei dati dalla tabella
    rows = []
    headers = ['Date', 'Ticker_added', 'Security_added', 'Ticker_removed', 'Security_removed', 'Reason']  # Estrae le intestazioni
    for row in table.find_all("tr")[2:]:  # Salta la riga delle intestazioni
        cells = row.find_all(["td", "th"])  # Prende sia td che th, nel caso di celle di intestazione
        rows.append([cell.text.strip() for cell in cells])

    # 6. Converti i dati in un DataFrame di Pandas (opzionale ma utile)
    df2 = pd.DataFrame(rows, columns=headers)

    # trasformo la data in un formato migliore
    df2['Date'] = pd.to_datetime(df2['Date'])
    #print(df2)
    # 7. Salva i dati in un file CSV
    df2.to_csv("sp500_companies_changes.csv", index=False)


else:
    print(f"Errore nella richiesta: {response.status_code}")


In [ ]:
import datetime as datetime

# Creo un data frame con la composizione dell'indice S&P 500 nel tempo a partire dai due csv

# carico i due file csv
df_companies = pd.read_csv("sp500_companies.csv")
df_changes = pd.read_csv("sp500_companies_changes.csv")

# creo un dizionario con le date e i ticker delle aziende
date_ticker_dict = {}
today = datetime.datetime.now().strftime('%Y-%m-%d')
# aggiungo le aziende presenti nel file df_companies con la data di oggi
current_composition = []
for index, row in df_companies.iterrows():
    current_composition.append(row['Symbol'])

c = 0
# aggiorno il dizionario con le aziende aggiunte e rimosse
print(current_composition)
for index, row in df_changes.iterrows():
    date_ticker_dict[row['Date']] = current_composition.copy()
    # prendo la composizione attuale e aggiorno in base ai cambiamenti
    if row['Ticker_removed'] is not None and pd.isna(row['Ticker_removed']) == False:
        #current_composition = [x for x in current_composition if x != row['Ticker_removed']]
        current_composition.append(row['Ticker_removed'])
    if row['Ticker_added'] is not None and pd.isna(row['Ticker_added']) == False:
        if row['Ticker_added'] not in current_composition:
            print(row['Ticker_added'])
            c += 1
        else:
            current_composition.remove(row['Ticker_added'])
    current_composition = current_composition.copy()

print(c)
# creo un data frame con la composizione dell'indice S&P 500 nel tempo
df_composition = pd.DataFrame(date_ticker_dict.items(), columns=['Date', 'Composition'])

#scrivo il file csv
df_composition.to_csv("sp500_composition.csv", index=False)

In [5]:
import pandas as pd

def check_symbol_availability(symbol):
    try:
        ticker = yf.Ticker(symbol)
        # Prova a ottenere i dati storici per vedere se il simbolo è valido
        data = ticker.history(period="1d")  # Richiedi un dato giornaliero
        if data.empty:
            return False  # Se i dati sono vuoti, il simbolo potrebbe non essere valido
        return True  # Il simbolo è valido e ha dati disponibili
    except Exception as e:
        print(f"Errore durante il controllo per {symbol}: {e}")
        return False
    

# prendo il csv
stock_processed = []
stock_not_available = []
df_composition = pd.read_csv("sp500_composition.csv")
#itero sulle righe
for index, row in df_composition.iterrows():
    print(row['Date'])
    #itero sui ticker
    for ticker in row['Composition'].split(","):
        ticker = ticker.replace("[", "").replace("]", "").replace(" ", "").replace("'", "")
        if ticker not in stock_processed:
            stock_processed.append(ticker)
            if check_symbol_availability(ticker) == False:
                print(f"Ticker {ticker} non disponibile")
                stock_not_available.append(ticker)

print(stock_not_available)
print(len(stock_not_available))

2024-10-01
Errore durante il controllo per MMM: name 'yf' is not defined
Ticker MMM non disponibile
Errore durante il controllo per AOS: name 'yf' is not defined
Ticker AOS non disponibile
Errore durante il controllo per ABT: name 'yf' is not defined
Ticker ABT non disponibile
Errore durante il controllo per ABBV: name 'yf' is not defined
Ticker ABBV non disponibile
Errore durante il controllo per ACN: name 'yf' is not defined
Ticker ACN non disponibile
Errore durante il controllo per ADBE: name 'yf' is not defined
Ticker ADBE non disponibile
Errore durante il controllo per AMD: name 'yf' is not defined
Ticker AMD non disponibile
Errore durante il controllo per AES: name 'yf' is not defined
Ticker AES non disponibile
Errore durante il controllo per AFL: name 'yf' is not defined
Ticker AFL non disponibile
Errore durante il controllo per A: name 'yf' is not defined
Ticker A non disponibile
Errore durante il controllo per APD: name 'yf' is not defined
Ticker APD non disponibile
Errore dur

In [3]:
import yfinance as yf

# Scarica i dati per l'indice S&P 500
sp500 = yf.Ticker("AMD")

# Dati storici dell'S&P 500 (per esempio, degli ultimi 5 anni)
data = sp500.history(period="max")
#filtro i dati e tengo solo 1 per mese
#data = data.resample('ME').last()

# scrivo su csv
data.to_csv('stock.csv')

In [4]:
#prendo tutte le stock singole all'interno del file csv

import pandas as pd
import yfinance

# Carica il file CSV con i ticker delle singole azioni
df = pd.read_csv("sp500_companies.csv")
df2 = pd.read_csv("sp500_companies_changes.csv")

# Crea un elenco di ticker univoci
tickers = df["Symbol"].tolist()
tickers2 = df2["Ticker_added"].tolist()

#unisco i due insiemi
tickers = list(set(tickers + tickers2))

print(tickers)
print(len(tickers))

#scrivo i dati su file csv
df = pd.DataFrame(tickers, columns=["Ticker"])
df.to_csv("sp500_tickers.csv", index=False)

['AEE', 'META', 'CF', 'BLK', 'JOYG', 'PTC', 'CL', 'GOOG', 'MJN', 'ATO', 'DECK', 'JBL', 'RF', 'TT', 'MNK', 'FB', 'HWM', 'LEN', 'TTWO', 'ALB', 'ALXN', 'DOW', 'AES', 'CVS', 'EPAM', 'IPG', 'ENDP', 'CNC', 'POOL', 'TER', 'EQIX', 'GE', 'NDAQ', 'KMX', 'TROW', 'JCI', 'JKHY', 'ODFL', 'NKTR', 'LW', 'HUM', 'AFL', 'COV', 'ULTA', 'IR', 'NDSN', 'NWSA', 'MDT', 'TAP', 'BALL', 'LULU', 'DISCK', 'GOOGL', 'LH', 'CNP', 'XYL', 'DFS', 'EFX', 'ITW', 'WLTW', 'PKG', 'COR', 'FOXA', 'MPC', 'HST', 'AME', 'CVX', 'GIS', 'MA', 'PFG', 'RCL', 'NI', 'ARE', 'WYNN', 'GEHC', 'MOH', 'JPM', 'ABNB', 'MEE', 'SJM', 'TWTR', 'EOG', 'MET', 'MXIM', 'CTRA', 'ENPH', 'CZR', 'MAC', 'LO', 'SCHW', 'CSRA', 'CHD', 'URI', 'EBAY', 'TFX', 'STE', 'GWW', 'DXCM', 'CHTR', 'WM', 'EXPD', 'ED', 'WST', 'APA', 'NSC', 'OMC', 'SYY', 'EQR', 'CLF', 'AMG', 'GD', 'MTCH', 'KRFT', 'COF', 'AMTM', 'ADSK', 'CPT', 'BXLT', 'EW', 'DAL', 'DRE', 'RTX', 'YHOO', 'ABMD', 'INTC', 'ANET', 'UDR', 'L', 'NAVI', 'LUK', 'NTRS', 'PENN', 'DD', 'PLTR', 'RJF', 'STZ', 'HSIC', 'PAYC'

In [8]:
# leggo il file csv con i ticker
df = pd.read_csv("sp500_tickers.csv")
tickers = df["Ticker"].tolist()

not_available = []

for ticker in tickers:
    try:
        data = yfinance.download(ticker, start=start_date_str, end=end_date_str)
        data.to_csv(f"stocks/{ticker}.csv")
    except Exception as e:
        print(f"Errore durante il download di {ticker}: {e}")
        not_available.append(ticker)

print(not_available)
print(len(not_available))
#scrivo su csv

df = pd.DataFrame(not_available, columns=["Ticker"])
df.to_csv("not_available_tickers.csv", index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['JOYG']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 1997-06-16 -> 2024-11-14)')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MJN']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 1997-06-16 -> 2024-11-14)')
[*********************100%***********************]  1 of 1 completed
[*********************100%***************

Errore durante il download di nan: 'float' object has no attribute 'replace'


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SIVB']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 1997-06-16 -> 2024-11-14)')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[nan]
1
